In [ ]:
!pip install -r requirements.txt

In [ ]:
# Install librosa (seems to come with ffmpeg as well):
!conda install -c conda-forge librosa -y

In [ ]:
# Install ffmpeg:
!conda install -c conda-forge ffmpeg -y

In [ ]:
# Copy files over from S3
!aws s3 cp s3://video-wav2lip/lessonVid.mp4 sample_data/
!aws s3 cp s3://video-wav2lip/video.mp4 sample_data/
!ls sample_data

In [ ]:
# Convert lesson video to audio
!ffmpeg -i sample_data/lessonVid.mp4 sample_data/lessonAudio.mp3 -y

In [ ]:
# If there is an error here, no output file will be generated!
!python inference.py --checkpoint_path checkpoints/wav2lip.pth --face "sample_data/video.mp4" --audio "sample_data/lessonAudio.mp3" --resize_factor 2

In [ ]:
# import os
# from email_aws import send_email 

# # If the video generation failed, call send email
# if not os.path.isfile("/results/result_voice.mp4"):
#     send_email(error=True)

### Moviepy Section

In [1]:
from moviepy_merge import merge_videos
from email_aws import send_email

merged_videos = merge_videos()
send_email(is_lesson=True) if merged_videos else send_email(error=True)

320 180
Moviepy - Building video ./results/result_lesson.mp4.
MoviePy - Writing audio in temp-audio.m4a


MoviePy - Done.
Moviepy - Writing video ./results/result_lesson.mp4



Moviepy - Done !
Moviepy - video ready ./results/result_lesson.mp4
VIDEOS MERGED
EMAIL SENT


### Clean Up

1. Remove Wav2Lip generated video
2. Remove Moviepy merged video
3. Remove user-uploaded face video from Amazon S3
4. Remove user-uploaded lesson video from Amazon S3

In [ ]:
!rm results/result_voice.mp4

In [ ]:
!rm results/result_lesson.mp4

In [ ]:
!aws s3 rm s3://video-wav2lip/video.mp4 

In [ ]:
!aws s3 rm s3://video-wav2lip/lessonVid.mp4 